In [1]:
!pip install matplotlib

In [2]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [3]:
!pip install torch==2.0.0 torchtext==0.15.1

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

In [5]:
!pip install torchdata==0.6.0

In [6]:
!pip install portalocker>=2.0.0

In [7]:
!pip install numpy==1.23.5

In [8]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 42.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

"""
Purpose: Defines special tokens used in the vocabulary for machine learning tasks with text data.
Special Tokens:
<unk>: "Unknown" token (represents words not in the vocabulary)
<pad>: Padding token (to make sequences the same length)
<bos>: "Beginning of Sequence"
<eos>: "End of Sequence"
"""

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

# Seq2Seq Network using Transformer

In [10]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [11]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [12]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [13]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [14]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [15]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:297: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.344, Val loss: 4.117, Epoch time = 10.569s
Epoch: 2, Train loss: 3.768, Val loss: 3.326, Epoch time = 10.161s
Epoch: 3, Train loss: 3.165, Val loss: 2.892, Epoch time = 10.159s
Epoch: 4, Train loss: 2.770, Val loss: 2.634, Epoch time = 10.014s
Epoch: 5, Train loss: 2.481, Val loss: 2.448, Epoch time = 10.158s
Epoch: 6, Train loss: 2.249, Val loss: 2.313, Epoch time = 10.062s
Epoch: 7, Train loss: 2.058, Val loss: 2.195, Epoch time = 10.259s
Epoch: 8, Train loss: 1.895, Val loss: 2.116, Epoch time = 10.225s
Epoch: 9, Train loss: 1.755, Val loss: 2.052, Epoch time = 10.080s
Epoch: 10, Train loss: 1.633, Val loss: 2.003, Epoch time = 10.278s
Epoch: 11, Train loss: 1.523, Val loss: 1.971, Epoch time = 10.136s
Epoch: 12, Train loss: 1.420, Val loss: 1.966, Epoch time = 10.112s
Epoch: 13, Train loss: 1.335, Val loss: 1.956, Epoch time = 10.150s
Epoch: 14, Train loss: 1.250, Val loss: 1.919, Epoch time = 10.111s
Epoch: 15, Train loss: 1.168, Val loss: 1.900, Epoch time

In [16]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu"))

 A group of people stand in front of an igloo 


In [17]:
print(translate(transformer, "Zwei Hunde schlafen ."))

 Two dogs are sleeping . 


# English to German


In [18]:
# Updated model names and variables for English-to-German translation
SRC_LANG_EN_TO_DE = 'en'
TGT_LANG_EN_TO_DE = 'de'

# Place-holders
token_map_en_to_de = {}
vocab_map_en_to_de = {}

# Tokenization setup
token_map_en_to_de[SRC_LANG_EN_TO_DE] = get_tokenizer('spacy', language='en_core_web_sm')
token_map_en_to_de[TGT_LANG_EN_TO_DE] = get_tokenizer('spacy', language='de_core_news_sm')

# Helper function for token yielding
def generate_tokens_en_to_de(data_iter: Iterable, lang: str) -> List[str]:
    lang_index = {SRC_LANG_EN_TO_DE: 0, TGT_LANG_EN_TO_DE: 1}

    for data_item in data_iter:
        yield token_map_en_to_de[lang](data_item[lang_index[lang]])

# Build vocab for English-to-German first
for ln in [SRC_LANG_EN_TO_DE, TGT_LANG_EN_TO_DE]:
    train_iter_en_to_de = Multi30k(split='train', language_pair=(SRC_LANG_EN_TO_DE, TGT_LANG_EN_TO_DE))
    vocab_map_en_to_de[ln] = build_vocab_from_iterator(
        generate_tokens_en_to_de(train_iter_en_to_de, ln),
        min_freq=1,
        specials=special_symbols,
        special_first=True
    )

# Set default index for unknown tokens in the vocab
for ln in [SRC_LANG_EN_TO_DE, TGT_LANG_EN_TO_DE]:
    vocab_map_en_to_de[ln].set_default_index(UNK_IDX)

# Now define `transform_text_en_to_de` after the vocab is ready
transform_text_en_to_de = {}
for ln in [SRC_LANG_EN_TO_DE, TGT_LANG_EN_TO_DE]:
    transform_text_en_to_de[ln] = sequential_transforms(
        token_map_en_to_de[ln],  # Tokenization
        vocab_map_en_to_de[ln],  # Numericalization
        tensor_transform  # Add BOS/EOS and create tensor
    )

# Updated transformer class for English-to-German
class TransformerModelEnToDe(nn.Module):
    def __init__(self,
                 encoder_layers: int,
                 decoder_layers: int,
                 embedding_size: int,
                 num_heads: int,
                 source_vocab_size: int,
                 target_vocab_size: int,
                 feedforward_dim: int = 512,
                 dropout_rate: float = 0.1):
        super(TransformerModelEnToDe, self).__init__()
        self.transformer = Transformer(d_model=embedding_size,
                                       nhead=num_heads,
                                       num_encoder_layers=encoder_layers,
                                       num_decoder_layers=decoder_layers,
                                       dim_feedforward=feedforward_dim,
                                       dropout=dropout_rate)
        self.output_layer = nn.Linear(embedding_size, target_vocab_size)
        self.src_embedding = TokenEmbedding(source_vocab_size, embedding_size)
        self.tgt_embedding = TokenEmbedding(target_vocab_size, embedding_size)
        self.positional_encoding = PositionalEncoding(embedding_size, dropout=dropout_rate)

    def forward(self,
                source: Tensor,
                target: Tensor,
                source_mask: Tensor,
                target_mask: Tensor,
                source_padding_mask: Tensor,
                target_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        source_emb = self.positional_encoding(self.src_embedding(source))
        target_emb = self.positional_encoding(self.tgt_embedding(target))
        outputs = self.transformer(source_emb, target_emb, source_mask, target_mask, None,
                                   source_padding_mask, target_padding_mask, memory_key_padding_mask)
        return self.output_layer(outputs)

    def encode(self, source: Tensor, source_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
            self.src_embedding(source)), source_mask)

    def decode(self, target: Tensor, memory: Tensor, target_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
            self.tgt_embedding(target)), memory, target_mask)

# Initialize the English-to-German transformer
SOURCE_VOCAB_SIZE_EN_TO_DE = len(vocab_map_en_to_de[SRC_LANG_EN_TO_DE])
TARGET_VOCAB_SIZE_EN_TO_DE = len(vocab_map_en_to_de[TGT_LANG_EN_TO_DE])
transformer_model_en_to_de = TransformerModelEnToDe(
    NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SOURCE_VOCAB_SIZE_EN_TO_DE, TARGET_VOCAB_SIZE_EN_TO_DE, FFN_HID_DIM
)

# Initialize parameters
for param in transformer_model_en_to_de.parameters():
    if param.dim() > 1:
        nn.init.xavier_uniform_(param)

transformer_model_en_to_de = transformer_model_en_to_de.to(DEVICE)

loss_function_en_to_de = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer_model_en_to_de = torch.optim.Adam(transformer_model_en_to_de.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# Updated collate function for English-to-German
def collate_batch_en_to_de(batch):
    source_batch, target_batch = [], []
    for source_item, target_item in batch:
        source_batch.append(transform_text_en_to_de[SRC_LANG_EN_TO_DE](source_item.rstrip("\n")))
        target_batch.append(transform_text_en_to_de[TGT_LANG_EN_TO_DE](target_item.rstrip("\n")))

    source_batch = pad_sequence(source_batch, padding_value=PAD_IDX)
    target_batch = pad_sequence(target_batch, padding_value=PAD_IDX)
    return source_batch, target_batch

# Training and evaluation loops for English-to-German
def train_loop_en_to_de(model, optimizer):
    model.train()
    total_loss = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANG_EN_TO_DE, TGT_LANG_EN_TO_DE))
    train_loader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_batch_en_to_de)

    for source, target in train_loader:
        source = source.to(DEVICE)
        target = target.to(DEVICE)

        target_input = target[:-1, :]

        source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(source, target_input)

        predictions = model(source, target_input, source_mask, target_mask, source_padding_mask, target_padding_mask, source_padding_mask)

        optimizer.zero_grad()

        target_output = target[1:, :]
        loss = loss_function_en_to_de(predictions.reshape(-1, predictions.shape[-1]), target_output.reshape(-1))
        loss.backward()

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(list(train_loader))


def eval_loop_en_to_de(model):
    model.eval()
    total_loss = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANG_EN_TO_DE, TGT_LANG_EN_TO_DE))
    val_loader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_batch_en_to_de)

    for source, target in val_loader:
        source = source.to(DEVICE)
        target = target.to(DEVICE)

        target_input = target[:-1, :]

        source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(source, target_input)

        predictions = model(source, target_input, source_mask, target_mask, source_padding_mask, target_padding_mask, source_padding_mask)

        target_output = target[1:, :]
        loss = loss_function_en_to_de(predictions.reshape(-1, predictions.shape[-1]), target_output.reshape(-1))
        total_loss += loss.item()

    return total_loss / len(list(val_loader))

# Translate function for English-to-German
def perform_translation_en_to_de(model: torch.nn.Module, source_sentence: str):
    model.eval()
    source = transform_text_en_to_de[SRC_LANG_EN_TO_DE](source_sentence).view(-1, 1)
    num_tokens = source.shape[0]
    source_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    target_tokens = greedy_decode1(
        model, source, source_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX
    ).flatten()
    return " ".join(vocab_map_en_to_de[TGT_LANG_EN_TO_DE].lookup_tokens(list(target_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

# Number of epochs for English-to-German
EPOCHS_EN_TO_DE = 13

# Training loop for English-to-German
for epoch in range(1, EPOCHS_EN_TO_DE + 1):
    start_time = timer()
    training_loss = train_loop_en_to_de(transformer_model_en_to_de, optimizer_model_en_to_de)
    end_time = timer()
    validation_loss = eval_loop_en_to_de(transformer_model_en_to_de)
    print(
        f"Epoch: {epoch}, Train loss: {training_loss:.3f}, Val loss: {validation_loss:.3f}, "
        f"Epoch time = {(end_time - start_time):.3f}s"
    )


Epoch: 1, Train loss: 5.902, Val loss: 4.677, Epoch time = 11.390s
Epoch: 2, Train loss: 4.138, Val loss: 3.765, Epoch time = 11.129s
Epoch: 3, Train loss: 3.464, Val loss: 3.311, Epoch time = 11.025s
Epoch: 4, Train loss: 3.024, Val loss: 3.014, Epoch time = 11.237s
Epoch: 5, Train loss: 2.700, Val loss: 2.811, Epoch time = 11.126s
Epoch: 6, Train loss: 2.451, Val loss: 2.668, Epoch time = 11.248s
Epoch: 7, Train loss: 2.241, Val loss: 2.526, Epoch time = 11.028s
Epoch: 8, Train loss: 2.065, Val loss: 2.425, Epoch time = 10.884s
Epoch: 9, Train loss: 1.914, Val loss: 2.332, Epoch time = 11.279s
Epoch: 10, Train loss: 1.779, Val loss: 2.244, Epoch time = 10.988s
Epoch: 11, Train loss: 1.660, Val loss: 2.210, Epoch time = 10.932s
Epoch: 12, Train loss: 1.553, Val loss: 2.192, Epoch time = 10.958s
Epoch: 13, Train loss: 1.459, Val loss: 2.167, Epoch time = 10.913s


In [19]:
def greedy_decode1(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.output_layer(out[:, -1])  # Use output_layer here
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).fill_(next_word).type(torch.long).to(DEVICE)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# Comparing the results

In [20]:
x= perform_translation_en_to_de(transformer_model_en_to_de, "This dog is sleeping . ")
x

' Ein Hund schläft . '

In [21]:
print(translate(transformer, x))

 A dog is sleeping . 


In [22]:
y= perform_translation_en_to_de(transformer_model_en_to_de, "A group of people stand in front of an igloo")
y

' Eine Gruppe von Menschen steht vor einem Essensstand . '

In [23]:
print(translate(transformer, y))

 A group of people stand in front of a food food stand . 


In [24]:
x1= perform_translation_en_to_de(transformer_model_en_to_de, "Cat is running")
x1

' Ein Sportler läuft . '

In [25]:
print(translate(transformer, x1))

 A athlete is running . 


In [26]:
x2= perform_translation_en_to_de(transformer_model_en_to_de, "Children  are playing in the park")
x2

' Kinder spielen im Park . '

In [27]:
print(translate(transformer, x2))

 There are kids playing in the park . 


If you see the output of x1. and x2 its different from the input. Where we see x1, output is tottle differnet and 